# MNIST convolutional neural networks with slim

* MNIST data를 가지고 **convolutional neural networks**를 `tf.contrib.slim`을 이용하여 만들어보자.
  * [참고: TensorFlow slim](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/slim)
* `tf.contrib.slim`에서 각 `layer`의 옵션들을 직접 컨트롤 해보자.
* `l2_regularization`을 사용해보자.
* `batch_norm`을 어떻게 사용하는지 알아보자.

### Import modules

In [ ]:
"""A deep MNIST classifier using convolutional layers.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/pros
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

slim = tf.contrib.slim

tf.set_random_seed(219)

### Import data

In [ ]:
data_dir = '../mnist'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

### Show the MNIST

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

index = 1000
print("label = ", np.argmax(mnist.train.labels[index]))
plt.imshow(mnist.train.images[index].reshape(28, 28), cmap='gray')
plt.show()

### Create the model

* [`tf.contrib.layers`](https://www.tensorflow.org/api_guides/python/contrib.layers)
* [`tf.contrib.layers.conv2d`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/conv2d)
* [`tf.contrib.layers.max_pool2d`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/max_pool2d)

In [ ]:
def deepnn_slim(x):
  is_training = tf.placeholder(tf.bool)
  batch_norm_params = {'decay': 0.9,
                       'epsilon': 0.001,
                       'is_training': is_training,
                       'scope': 'batch_norm'}
  l2_decay = 0.0001
  
  # Reshape to use within a convolutional neural net.
  # Last dimension is for "features" - there is only one here, since images are
  # grayscale -- it would be 3 for an RGB image, 4 for RGBA, etc.
  with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

  # First convolutional layer - maps one grayscale image to 32 feature maps.
  conv1 = slim.conv2d(inputs=x_image,
                      num_outputs=32,
                      kernel_size=[5, 5],
                      stride=[1, 1],
                      padding='SAME',
                      activation_fn=tf.nn.relu,
                      normalizer_fn=slim.batch_norm,
                      normalizer_params=batch_norm_params,
                      weights_initializer=slim.xavier_initializer(),
                      weights_regularizer=slim.l2_regularizer(l2_decay),
                      scope='conv1')
    
  # Pooling layer - downsamples by 2X.
  pool1 = slim.max_pool2d(inputs=conv1,
                          kernel_size=[2, 2],
                          stride=2,
                          padding='VALID',
                          scope='pool1')

  # Second convolutional layer -- maps 32 feature maps to 64.
  conv2 = slim.conv2d(inputs=pool1,
                      num_outputs=64,
                      kernel_size=[5, 5],
                      stride=[1, 1],
                      padding='SAME',
                      activation_fn=tf.nn.relu,
                      normalizer_fn=slim.batch_norm,
                      normalizer_params=batch_norm_params,
                      weights_initializer=slim.xavier_initializer(),
                      weights_regularizer=slim.l2_regularizer(l2_decay),
                      scope='conv2')
    
  # Second pooling layer.
  pool2 = slim.max_pool2d(inputs=conv2,
                          kernel_size=[2, 2],
                          stride=2,
                          padding='VALID',
                          scope='pool2')

  # Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
  # is down to 7x7x64 feature maps -- maps this to 1024 features.
  flat = slim.flatten(pool2, scope='flatten')

  fc1 = slim.fully_connected(flat,
                             num_outputs=1024,
                             activation_fn=tf.nn.relu,
                             normalizer_fn=slim.batch_norm,
                             normalizer_params=batch_norm_params,
                             weights_initializer=slim.xavier_initializer(),
                             weights_regularizer=slim.l2_regularizer(l2_decay),
                             scope='fc1')

  # Dropout - controls the complexity of the model, prevents co-adaptation of
  # features.
  #with tf.name_scope('dropout'):
  #keep_prob = tf.placeholder(tf.float32)
  #fc1_drop = slim.dropout(fc1, keep_prob)
  fc1_drop = slim.dropout(fc1, keep_prob=0.5, is_training=is_training, scope='dropout')

  # Map the 1024 features to 10 classes, one for each digit
  fc2 = slim.fully_connected(fc1_drop,
                             num_outputs=10,
                             activation_fn=None,
                             normalizer_fn=slim.batch_norm,
                             normalizer_params=batch_norm_params,
                             weights_initializer=slim.xavier_initializer(),
                             weights_regularizer=slim.l2_regularizer(l2_decay),
                             scope='fc2')
    
  return fc2, is_training

### Create the placeholder

In [ ]:
# Only necessary if you use IDLE or a jupyter notebook
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

### Build the model and define loss and optimizer

In [ ]:
# Build the graph for the deep net
y_conv, is_training = deepnn_slim(x)

cross_entropy = tf.losses.softmax_cross_entropy(onehot_labels=y_,
                                                logits=y_conv,
                                                scope='cross_entropy')
l2_regualrization = tf.reduce_sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
with tf.name_scope('total_loss'):
  total_loss = cross_entropy + l2_regualrization


#with tf.name_scope('adam_optimizer'):
#  train_step = tf.train.AdamOptimizer(1e-4).minimize(total_loss)

with tf.name_scope('accuracy'):
  correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
  correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)

graph_location = 'graphs/code13_mnist_slim_options'
print('Saving graph to: %s' % graph_location)
train_writer = tf.summary.FileWriter(graph_location)
train_writer.add_graph(tf.get_default_graph())  

In [ ]:
tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)

### Batch normalization update

In [ ]:
tf.get_collection(tf.GraphKeys.UPDATE_OPS)

In [ ]:
# Batch normalization update
batchnorm_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

# Add dependency to compute batchnorm_updates.
with tf.control_dependencies(batchnorm_update_ops):
  train_step = tf.train.AdamOptimizer(1e-4).minimize(total_loss)

### tf.Session() and train

In [ ]:
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=sess_config)
sess.run(tf.global_variables_initializer())

max_step = 100
for step in range(max_step+1):
  batch = mnist.train.next_batch(32)
  feed_dict_train = {x: batch[0],
                     y_: batch[1],
                     is_training: True}
  feed_dict_eval = {x: batch[0],
                    y_: batch[1],
                    is_training: False}
  _, tl, ce = sess.run([train_step, total_loss, cross_entropy], feed_dict=feed_dict_train)
  if step % 10 == 0:
    train_accuracy = sess.run(accuracy, feed_dict=feed_dict_eval)
    print('step %d, training accuracy %g, total_loss %g, cross_entropy %g' % (step, train_accuracy, tl, ce))

### Test trained model

* test accuracy: 0.9378

In [ ]:
print('test accuracy %g' % sess.run(accuracy, feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, is_training: False}))

### test images inference

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

test_batch_size = 16
batch_xs, _ = mnist.test.next_batch(test_batch_size)
y_pred = sess.run(y_conv, feed_dict={x: batch_xs,
                                     is_training: False})
  
fig = plt.figure(figsize=(16, 10))
for i, (px, py) in enumerate(zip(batch_xs, y_pred)):
  p = fig.add_subplot(4, 8, i+1)
  p.set_title("y_pred: {}".format(np.argmax(py)))
  p.imshow(px.reshape(28, 28), cmap='gray')
  p.axis('off')

### Print all trainable variables

In [ ]:
tf.trainable_variables()

In [ ]:
tf.global_variables()

In [ ]:
tf.model_variables()